# Bingo!

![game](http://s2.quickmeme.com/img/14/1494d43f99b56ab589c88a8138175c488be95d4c0b4dbdaebdea05d38790055f.jpg)

In [1]:
import pandas
import itertools
import operator
import numpy as np

NUM_WINNERS = 2
SEED = 42
NUM_DIGITS = 4

In [2]:
emails = pandas.read_csv("emails.csv")
print emails.count()
emails.head()

email    3
dtype: int64


,email
0,java@otus.ru
1,python@otus.ru
2,guido@otus.ru


# Naive approach

In [3]:
emails.sample(NUM_WINNERS, random_state=SEED)

,email
0,java@otus.ru
1,python@otus.ru


# Enter Pi

In [4]:
# http://www.cs.ox.ac.uk/people/jeremy.gibbons/publications/spigot.pdf
def pi_digits():
    """generator for digits of pi"""
    q, r, t, k, n, l = 1, 0, 1, 1, 3, 3
    while True:
        if 4 * q + r - t < n * t:
            yield n
            q, r, t, k, n, l = (10*q, 10*(r-n*t), t, k, (10*(3*q+r))/t-10*n, l)
        else:
            q, r, t, k, n, l = (q*k, (2*q+r)*l, t*l, k+1, (q*(7*k+2)+r*l)/(t*l), l+2)


_g = pi_digits()
print "".join([str(_g.next()) for _ in xrange(20)])

31415926535897932384


In [5]:
np.random.seed(SEED)
emails["num"] = np.random.randint(10 ** (NUM_DIGITS - 1), 10 ** NUM_DIGITS - 1, size=len(emails))
assert len(emails["email"].unique()) ==  len(emails["num"].unique())
emails.head()

,email,num
0,java@otus.ru,8270
1,python@otus.ru,1860
2,guido@otus.ru,6390


In [9]:
class _Num(object):
    def __init__(self, n):
        self.n = n
        self.s = str(n)
        self.p = 0 # pointer in number string representation
        self.l = len(self.s)

    def move_p(self, d):
        if d == self.s[self.p]:
            self.p += 1
        else:
            self.p = 0

    
def find_nums_in_pi(nums, first_n=None):
    MAX_POS = 10 ** 6
    pi_gen = pi_digits()
    first_n = first_n if first_n is not None else len(nums)
    _nums = [_Num(n) for n in nums]
    nums_pos = {}
    for pos in itertools.count():
        if pos % 1000 == 0:
            print "Current Pi position: %s. Nums found: %s" % (pos, len(nums_pos))
        if pos == MAX_POS:
            raise RuntimeError("Circuit breaker!")
        d = str(pi_gen.next())
        found_num = None
        for cur_num in _nums:
            cur_num.move_p(d)
            # whole number found
            if cur_num.p == cur_num.l:
                nums_pos[cur_num.n] = pos - cur_num.l + 1
                # found enough numbers
                if len(nums_pos) == first_n:
                    return nums_pos                
                found_num = cur_num.n
        # create new search array without found number
        if found_num:
            _nums = [num for num in _nums if num.n != found_num]

# regression test
assert find_nums_in_pi([15, 59, 92]) == {59: 4, 92: 5, 15: 3}

Current Pi position: 0. Nums found: 0


In [7]:
%%time
num_pos = find_nums_in_pi(emails["num"].tolist(), first_n=NUM_WINNERS)
print num_pos

Current Pi position: 0. Nums found: 0
Current Pi position: 1000. Nums found: 0
Current Pi position: 2000. Nums found: 0
Current Pi position: 3000. Nums found: 0
Current Pi position: 4000. Nums found: 0
Current Pi position: 5000. Nums found: 0
Current Pi position: 6000. Nums found: 0
Current Pi position: 7000. Nums found: 0
Current Pi position: 8000. Nums found: 1
{1860: 8863, 6390: 7360}
CPU times: user 7.45 s, sys: 150 ms, total: 7.6 s
Wall time: 8.1 s


In [8]:
for n, p in sorted(num_pos.iteritems(), key=operator.itemgetter(1)):
    r = emails.loc[emails['num'] == n]
    print "%s -> %s" % (r["email"].iloc[0], p)

guido@otus.ru -> 7360
python@otus.ru -> 8863


![winner](http://podrobnosti.ua/media/pictures/2015/10/30/thumbs/740x415/v-etu-pjatnitsu-pole-chudes-vyhodit-v-efir-rovno-25-god-podrjad-kadr-iz-video_rect_8d40de7cfa137c6ac9d9fcd7378620c0.jpg)